# Music Generation with LSTM model

## Imports

In [15]:

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from music21 import converter, note, chord
import os
import pickle
import numpy as np

## Load Chopin MIDI files

In [3]:
data_folder = 'data'
files = [os.path.join(data_folder, file) for file 
         in os.listdir(data_folder) if file.endswith('.mid')]

In [4]:
files

['data/chpn-p4.mid',
 'data/chpn-p14.mid',
 'data/chpn-p3.mid',
 'data/chpn-p7.mid',
 'data/chpn_op66.mid',
 'data/chpn_op25_e1.mid',
 'data/chpn_op27_2.mid',
 'data/chpn_op25_e4.mid',
 'data/chpn_op7_2.mid',
 'data/chpn_op7_1.mid',
 'data/chpn-p10.mid',
 'data/chpn_op25_e3.mid',
 'data/chpn_op25_e2.mid',
 'data/chpn-p5.mid',
 'data/chpn_op35_4.mid',
 'data/chpn_op10_e05.mid',
 'data/chpn-p21.mid',
 'data/chpn-p15.mid',
 'data/chpn-p24.mid',
 'data/chpn_op35_1.mid',
 'data/chpn-p17.mid',
 'data/chpn-p6.mid',
 'data/chpn_op33_4.mid',
 'data/chpn-p11.mid',
 'data/chpn_op10_e12.mid',
 'data/chpn-p22.mid',
 'data/chpn_op10_e01.mid',
 'data/chpn-p23.mid',
 'data/chpn-p2.mid',
 'data/chpn-p18.mid',
 'data/chpn_op33_2.mid',
 'data/chpn-p13.mid',
 'data/chpn_op23.mid',
 'data/chpn_op35_2.mid',
 'data/chpn-p9.mid',
 'data/chpn_op53.mid',
 'data/chpn_op35_3.mid',
 'data/chp_op18.mid',
 'data/chpn-p19.mid',
 'data/chpn-p16.mid',
 'data/chpn-p1.mid',
 'data/chpn_op25_e12.mid',
 'data/chp_op31.mid'

In [7]:
def get_notes_from_the_files(files):
    """ Get all the notes and chords from the MIDI files in the specified directory """
    notes = []
    for file in files:
        print(f"{file} parsing...")
        midi = converter.parse(file)
        notes_to_parse = midi.flatten()
        # D C E -> D.C.E
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.pitches))
    return notes

In [8]:
notes_file_path = f'{data_folder}/notes'
notes = []

if os.path.exists(notes_file_path):
   # Loading notes from the file
   with open('data/notes', 'rb') as filepath:
       notes = pickle.load(filepath)
else:
   # creating a list of notes
   notes = get_notes_from_the_files(files)
   # saving notes to the file:
   with open('data/notes', 'wb') as filepath:
       pickle.dump(notes, filepath)

data/chpn-p4.mid parsing...
data/chpn-p14.mid parsing...
data/chpn-p3.mid parsing...
data/chpn-p7.mid parsing...
data/chpn_op66.mid parsing...
data/chpn_op25_e1.mid parsing...
data/chpn_op27_2.mid parsing...
data/chpn_op25_e4.mid parsing...
data/chpn_op7_2.mid parsing...
data/chpn_op7_1.mid parsing...
data/chpn-p10.mid parsing...
data/chpn_op25_e3.mid parsing...
data/chpn_op25_e2.mid parsing...
data/chpn-p5.mid parsing...
data/chpn_op35_4.mid parsing...
data/chpn_op10_e05.mid parsing...
data/chpn-p21.mid parsing...
data/chpn-p15.mid parsing...
data/chpn-p24.mid parsing...
data/chpn_op35_1.mid parsing...
data/chpn-p17.mid parsing...
data/chpn-p6.mid parsing...
data/chpn_op33_4.mid parsing...
data/chpn-p11.mid parsing...
data/chpn_op10_e12.mid parsing...
data/chpn-p22.mid parsing...
data/chpn_op10_e01.mid parsing...
data/chpn-p23.mid parsing...
data/chpn-p2.mid parsing...
data/chpn-p18.mid parsing...
data/chpn_op33_2.mid parsing...
data/chpn-p13.mid parsing...
data/chpn_op23.mid parsing.

In [10]:
notes[0:10]

['B3',
 'B4',
 'B4',
 'G3.B3.E4',
 'G3.B3.E4',
 'G3.B3.E4',
 'G3.B3.E4',
 'G3.B3.E4',
 'G3.B3.E4',
 'C5']

In [11]:
SEQUENCE_LENGTH = 100

In [12]:
# Create input sequences and corresponding output
unique_notes = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

input_sequences = []
output_sequences = []

for i in range(len(notes) - SEQUENCE_LENGTH):
    sequence_in = notes[i:i + SEQUENCE_LENGTH]
    sequence_out = notes[SEQUENCE_LENGTH]
    input_sequences.append([note_to_int[char] for char in sequence_in])
    output_sequences.append(note_to_int[sequence_out])

## Build and train LSTM model

In [13]:
EPOCHS = 100  # Adjust as needed
BATCH_SIZE = 128  # Adjust as needed

In [ ]:
# Reshape input sequences
X = np.reshape(input_sequences, (len(input_sequences), SEQUENCE_LENGTH, 1))
X = X / float(len(unique_notes))

# One-hot encode output sequences
y = to_categorical(output_sequences)

model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

checkpoint = ModelCheckpoint('model/', save_best_only=True, monitor='loss')

model.compile(loss='categorical_crossentropy', optimizer='adam')

check_point = ModelCheckpoint('model/', save_best_only=True, monitor='loss')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Train the model
model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[check_point])
